In [26]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

In [27]:
def extract_2d_array(tensor_str):
    
    tensor_str = tensor_str.replace('tensor(','').replace('grad_fn=<TanhBackward0>)', '').replace(' ', '').replace('\n', '')
    tensor_str = tensor_str[:len(tensor_str)-1]
    numbers = eval(tensor_str)
    
    array = np.array(numbers).reshape((-1, 768))  
    # print(array)
    
    return array

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


In [28]:
def constructFinalDataframe(path, bigger_path):

    files = glob.glob(path)
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1", 'Unnamed: 0.2'], errors="ignore")
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True)
    df_big = pd.read_csv(bigger_path)
    
    names = df['name']
    filtered_df = df_big[df_big['name'].isin(names)]
    df_unique = filtered_df.drop_duplicates(subset='name', keep='first')

    df_unique.to_csv("dataset_truncated.csv", index=False)

In [29]:
constructFinalDataframe("./final_csvs/*/*.csv", "dataset.csv")

In [30]:
df = pd.read_csv("./dataset_truncated.csv")

df = df[df['ratings'].apply(is_float) & df['no_of_ratings'].apply(is_float)]

df['ratings'] = df['ratings'].astype('float64')
df['no_of_ratings'] = df['no_of_ratings'].astype('int')

grouped = df.groupby(by=['sub_category'])
ratings_mean = grouped['ratings'].mean()
print(ratings_mean)
number_mean = grouped['no_of_ratings'].mean()
print(number_mean)

sub_category
Air Conditioners                            3.787563
All Appliances                              3.895865
All Car & Motorbike Products                3.823115
All Electronics                             4.015314
All Exercise & Fitness                      3.879464
All Grocery & Gourmet Foods                 4.155153
All Home & Kitchen                          4.051107
All Pet Supplies                            3.999612
All Sports, Fitness & Outdoors              3.860286
Amazon Fashion                              4.002215
Camping & Hiking                            4.075000
Car Accessories                             3.965957
Cardio Equipment                            3.866667
Casual Shoes                                3.500000
Clothing                                    3.788596
Cricket                                     3.933333
Cycling                                     3.809091
Ethnic Wear                                 3.753448
Fashion & Silver Jewellery       

In [31]:
# Group by 'sub_category' and calculate the 1/4th quartile for 'no_of_ratings'
quartile_1_no = grouped['no_of_ratings'].quantile(0.25)
# Print the 1/4th quartile values
print(quartile_1_no)

sub_category
Air Conditioners                              3.00
All Appliances                               15.00
All Car & Motorbike Products                 12.00
All Electronics                              63.00
All Exercise & Fitness                        8.00
All Grocery & Gourmet Foods                  81.50
All Home & Kitchen                           48.00
All Pet Supplies                             12.00
All Sports, Fitness & Outdoors               15.75
Amazon Fashion                               34.75
Camping & Hiking                             53.50
Car Accessories                               9.00
Cardio Equipment                             25.50
Casual Shoes                                 12.00
Clothing                                     11.00
Cricket                                      78.75
Cycling                                       4.25
Ethnic Wear                                  13.00
Fashion & Silver Jewellery                   33.25
Fashion Sandals   

In [32]:
# Group by 'sub_category' and calculate the 1/4th quartile for 'no_of_ratings'
quartile_1_rating = grouped['ratings'].quantile(0.75)
# Print the 1/4th quartile values
print(quartile_1_rating)

sub_category
Air Conditioners                            4.200
All Appliances                              4.200
All Car & Motorbike Products                4.200
All Electronics                             4.300
All Exercise & Fitness                      4.200
All Grocery & Gourmet Foods                 4.300
All Home & Kitchen                          4.300
All Pet Supplies                            4.350
All Sports, Fitness & Outdoors              4.200
Amazon Fashion                              4.200
Camping & Hiking                            4.150
Car Accessories                             4.300
Cardio Equipment                            3.900
Casual Shoes                                3.500
Clothing                                    4.100
Cricket                                     4.225
Cycling                                     4.275
Ethnic Wear                                 3.900
Fashion & Silver Jewellery                  4.100
Fashion Sandals                      

In [33]:
# # Create scatter plots for each sub_category
# for sub_category, group in grouped:
#     plt.figure(figsize=(8, 6))
#     plt.scatter(group['no_of_ratings'], group['ratings'], alpha=0.6)
#     plt.title(f'Scatter Plot for {sub_category}')
#     plt.xlabel('Number of Ratings')
#     plt.ylabel('Ratings')
#     plt.grid(True)
#     plt.show()

In [34]:
def apply_thresholds(row):
    category = row['sub_category']
    return (row['no_of_ratings'] < quartile_1_no[category]) and (row['ratings'] > quartile_1_rating[category])

df_filtered = df[df.apply(apply_thresholds, axis=1)]
print(df_filtered.shape)

(4140, 10)


In [35]:
df_filtered.drop(columns=['Unnamed: 0']).to_csv("Hidden products.csv", index=False)